## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 🏦 Azure AI Agent Basic Example 💼

This notebook demonstrates basic usage of `AzureAIProjectAgentProvider` for creating and using Azure AI Agents.

## Features Covered:
- Setting up Azure AI Project Agent Provider 
- Creating an Advisor agent with function tools
- Using function tools for banking operations
- Streaming and non-streaming responses
- Proper resource management with async context managers

### ⚠️ Important Financial Disclaimer ⚠️
> **The financial information provided by this notebook is for general educational and demonstration purposes only and is not intended as financial, investment, legal, or tax advice.** Always consult with qualified financial advisors before making any financial decisions.

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Microsoft Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details:
   - `AI_FOUNDRY_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

## Import Required Libraries

First, let's import all the necessary libraries and modules. We use `AzureAIProjectAgentProvider` :

In [ ]:
# Copyright (c) Microsoft. All rights reserved.

from random import randint, uniform
from typing import Annotated


from agent_framework.azure import AzureAIProjectAgentProvider
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Initial Setup

Load environment variables from the `.env` file for Azure AI Project configuration:

In [ ]:
from pathlib import Path
import os
from dotenv import load_dotenv

# Load environment variables from .env file (two directories up)
notebook_path = Path().absolute()
load_dotenv('../../.env')

# Verify environment setup
endpoint = os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')
model = os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print("🔧 Environment Configuration:")
print(f"✅ Project Endpoint: {endpoint[:50]}..." if endpoint else "❌ AI_FOUNDRY_PROJECT_ENDPOINT not set")
print(f"✅ Model Deployment: {model}" if model else "❌ AZURE_AI_MODEL_DEPLOYMENT_NAME not set")

## Define Function Tools 🏦

Function tools allow the agent to call specific functions to gather information or perform operations. Here we define functions for account balance and loan rate inquiries:

In [ ]:
def get_account_balance(
    account_id: Annotated[str, Field(description="The customer account ID to check balance for.")],
) -> str:
    """Get the current balance for a customer account."""
    # Simulated account balances for demo
    balances = {
        "checking": round(uniform(1000, 15000), 2),
        "savings": round(uniform(5000, 50000), 2),
        "investment": round(uniform(10000, 100000), 2)
    }
    account_type = ["checking", "savings", "investment"][randint(0, 2)]
    return f"Account {account_id} ({account_type}): Current balance is ${balances[account_type]:,.2f}"


def get_loan_rates(
    loan_type: Annotated[str, Field(description="Type of loan: mortgage, auto, personal, or business")],
) -> str:
    """Get current interest rates for different loan types."""
    rates = {
        "mortgage": {"rate": 6.5, "term": "30 years", "min_credit": 620},
        "auto": {"rate": 7.2, "term": "5 years", "min_credit": 600},
        "personal": {"rate": 10.5, "term": "3 years", "min_credit": 650},
        "business": {"rate": 8.0, "term": "10 years", "min_credit": 680}
    }
    loan_type = loan_type.lower()
    if loan_type in rates:
        r = rates[loan_type]
        return f"Current {loan_type} loan rates: {r['rate']}% APR, {r['term']} term, minimum credit score: {r['min_credit']}"
    return f"Unknown loan type: {loan_type}. Available types: mortgage, auto, personal, business"


# Financial disclaimer constant
FINANCIAL_DISCLAIMER = """
⚠️ DISCLAIMER: This information is for educational purposes only and does not constitute 
financial advice. Please consult with a qualified financial advisor for personalized guidance.
"""

## Non-Streaming Response Example 💰

In this example, we'll create a Financial Advisor agent using the new `AzureAIProjectAgentProvider` API and get a complete response at once (non-streaming):



- Use `await provider.create_agent()` instead of context manager pattern

In [ ]:
async def non_streaming_example() -> None:
    """Example of non-streaming response (get the complete result at once)."""
    print("=== 🏦 Non-streaming Response Example ===")


    # For authentication, run `az login` command in terminal
    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(credential=credential) as provider,
    ):
        # Create agent using provider.create_agent() instead of context manager
        agent = await provider.create_agent(
            name="FinancialAdvisorAgent",
            instructions="""You are a helpful Financial Services Advisor for a retail bank.
            You can help customers check account balances and inquire about loan rates.
            Always be professional, accurate, and remind customers that this is general information only.""",
            tools=[get_account_balance, get_loan_rates],
        )

        query = "What are the current mortgage loan rates?"
        print(f"🤔 Customer: {query}")
        result = await agent.run(query)
        print(f"🏦 Advisor: {result}")
        print(FINANCIAL_DISCLAIMER)

## Streaming Response Example 📊

In this example, we'll demonstrate streaming responses where we get results as they are generated by the Financial Advisor agent:

In [ ]:
async def streaming_example() -> None:
    """Example of streaming response (get results as they are generated)."""
    print("=== 📊 Streaming Response Example ===")


    async with (
        AzureCliCredential() as credential,
        AzureAIProjectAgentProvider(credential=credential) as provider,
    ):
        agent = await provider.create_agent(
            name="FinancialAdvisorAgent",
            instructions="""You are a helpful Financial Services Advisor for a retail bank.
            You can help customers check account balances and inquire about loan rates.
            Always be professional and provide clear, helpful information.""",
            tools=[get_account_balance, get_loan_rates],
        )

        query = "Can you check the balance for account ACC-12345 and also tell me about auto loan rates?"
        print(f"🤔 Customer: {query}")
        print("🏦 Advisor: ", end="", flush=True)
        async for chunk in agent.run_stream(query):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print("\n")
        print(FINANCIAL_DISCLAIMER)

## Main Execution Function

This function orchestrates the execution of both examples:

In [ ]:
async def main() -> None:
    print("=== 🏦 Azure AI Agent Example ===\n")
    
    await non_streaming_example()
    print("\n" + "="*60 + "\n")
    await streaming_example()

## Run the Examples 🚀

Execute the main function to run both streaming and non-streaming Financial Advisor examples:

In [ ]:
# Run the main function
await main()

## Key Takeaways 📚





| `client.create_agent()` as context manager | `await provider.create_agent()` returns agent |


### Key Concepts

1. **New Provider Pattern**: Use `AzureAIProjectAgentProvider(credential=credential)` as async context manager
2. **Agent Creation**: `agent = await provider.create_agent(name=..., instructions=..., tools=...)` 
3. **Function Tools**: Agents can use custom functions for banking operations
4. **Response Types**: 
   - Non-streaming: Get complete response at once using `agent.run()`
   - Streaming: Get response chunks as they're generated using `agent.run_stream()`
5. **Context Managers**: Using `async with` ensures proper resource cleanup
6. **Use Cases**: Applications like balance inquiries, loan rate checks

